In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from keras import optimizers

import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
# dimensions of our images.
img_width, img_height = 256, 256

train_data_dir = '/home/rte/data/images/random/seq/classes/train/'
validation_data_dir = '/home/rte/data/images/random/seq/classes/val/'
test_data_dir = '/home/rte/data/images/random/seq/classes/test/'
nb_train_samples = 1882
nb_validation_samples = 470
epochs = 5
batch_size = 16

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

In [ ]:
model.load_weights('first_try_diagram_sensor.h5')

In [ ]:
model.load_weights('first_try_unsure.h5')

In [ ]:
test_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

filenames = test_generator.filenames
nb_samples = len(filenames)

In [ ]:
probabilities = model.predict_generator(test_generator, nb_samples, verbose=True)
# steps = nb_samples

In [ ]:
p = np.array(probabilities)
np.set_printoptions(precision=3, suppress=True)
print(p)

In [ ]:
# broken
image = test_generator.next()
p = model.predict(image)
print(p)

In [ ]:
predictions = model.predict()

In [ ]:
import PIL
from PIL import Image
import os

list_of_labels = os.listdir(test_data_dir)
for label in list_of_labels:
    current_label_dir_path = os.path.join(test_data_dir, label)
    list_of_images = os.listdir(current_label_dir_path)
    for image in list_of_images:
        current_image_path = os.path.join(current_label_dir_path, image)
#         image = Image.open(current_image_path) # use the function which you want.
        print(current_image_path)

In [ ]:
for p in probabilities:
    print(p)

In [ ]:
print(probabilities)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]